In [1]:
import random
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

In [104]:
BATCH_SIZE = 32
CONTEXT_SIZE = 64
EMBEDDING_DIMS = 1024
HIDDEN_LAYERS = 5
HIDDEN_DIMS = 1024
EPOCHS = 100000
LR = 0.01

DEV = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [105]:
with open("data.txt") as f:
    data = f.read()
chars = list(set(data))
VOCAB_SIZE = len(chars)
s_to_i = {s: i for i, s in enumerate(chars)}
i_to_s = {v: k for k, v in s_to_i.items()}

def encode(text):
    return [s_to_i[c] for c in text]

def decode(ids):
    return "".join([i_to_s[i] for i in ids])

data = torch.tensor(encode(data), device=DEV)
train_data = data[:int(len(data) * 0.9)]
val_data = data[int(len(data) * 0.9):]

def get_batch(split):
    data = train_data if split == "train" else val_data
    idxs = [random.randint(0, data.shape[-1] - CONTEXT_SIZE - 1) for _ in range(BATCH_SIZE)]
    x = torch.stack([data[idx: idx + CONTEXT_SIZE] for idx in idxs])
    y = torch.stack([data[idx + 1: idx + 1 + CONTEXT_SIZE] for idx in idxs])
    return x, y

In [106]:
class FCL(nn.Module):
    def __init__(self, fan_in=HIDDEN_DIMS, fan_out=HIDDEN_DIMS):
        super().__init__()
        self.layers = [
            nn.Linear(fan_in, fan_out),
            nn.ReLU(inplace=True)
        ]
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [116]:
class LM(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMS),
            FCL(EMBEDDING_DIMS, HIDDEN_DIMS),
            *[FCL() for _ in range(HIDDEN_LAYERS)],
            nn.Linear(HIDDEN_DIMS, VOCAB_SIZE)
        )

    def forward(self, tokens):
        return self.layers(tokens)
    
    def generate(self, context="\n", new_tokens=100):
        context = torch.tensor([encode(context)])
        out = []
        while len(out) < new_tokens:
            logits = net(context)
            probs = F.softmax(logits, dim=-1)[:, -1:, :]
            next = torch.multinomial(probs.view(-1, VOCAB_SIZE), num_samples=1)
            out.append(next.item())
            print(decode([next.item()]), end="")
            context = torch.cat((context, next), dim=-1)
        return decode(out)
    
    
net = LM()
net.load_state_dict(torch.load("emb_1024_loss_2.5.pth"))

<All keys matched successfully>

In [108]:
@torch.no_grad()
def get_split_losses():
    net.eval()
    losses = {}
    for split in ["train", "val"]:
        x, y = get_batch(split)
        logits = net(x)
        loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), y.view(-1))
        losses[split] = loss.item()
    return losses.values()

In [109]:
print("%d trainable parameters" % (sum(p.nelement() for p in net.parameters())))
optimizer = optim.AdamW(net.parameters(), lr=LR)

for e in range(1, EPOCHS + 1):
    if e % (EPOCHS / 25) == 0:
        LR /= 2
        for group in optimizer.param_groups:
            group["lr"] = LR
    x, y, = get_batch("train")
    logits = net(x)
    loss = F.cross_entropy(logits.view(-1, logits.shape[-1] ), y.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    t_loss, v_loss = get_split_losses()
    print("%d: lr: %.4f, train loss: %.4f, val loss: %.4f" % (e, LR, t_loss, v_loss))

133185 trainable parameters
1: lr: 0.0100, train loss: 4.0716, val loss: 4.0722
2: lr: 0.0100, train loss: 3.9708, val loss: 3.9793
3: lr: 0.0100, train loss: 3.8757, val loss: 3.8892
4: lr: 0.0100, train loss: 3.8051, val loss: 3.8282
5: lr: 0.0100, train loss: 3.7193, val loss: 3.7384
6: lr: 0.0100, train loss: 3.6652, val loss: 3.6832
7: lr: 0.0100, train loss: 3.6139, val loss: 3.6301
8: lr: 0.0100, train loss: 3.5747, val loss: 3.6011
9: lr: 0.0100, train loss: 3.4988, val loss: 3.5275
10: lr: 0.0100, train loss: 3.5474, val loss: 3.5115
11: lr: 0.0100, train loss: 3.4255, val loss: 3.5448
12: lr: 0.0100, train loss: 3.4890, val loss: 3.5349
13: lr: 0.0100, train loss: 3.3814, val loss: 3.4404
14: lr: 0.0100, train loss: 3.4319, val loss: 3.3588
15: lr: 0.0100, train loss: 3.3284, val loss: 3.3677
16: lr: 0.0100, train loss: 3.3589, val loss: 3.3803
17: lr: 0.0100, train loss: 3.2690, val loss: 3.3708
18: lr: 0.0100, train loss: 3.2924, val loss: 3.3691
19: lr: 0.0100, train loss:

KeyboardInterrupt: 

In [124]:
net.generate(new_tokens=100000)

ouoa  oa ieu a aeoo  iuloioe ooau:eoul ooie oeeeo o  oreo o:ou oi e  oeo ll   e    oo  ueaoi  ooeeoeulioaaoeo:eooaoeei oul::aaooua aoiu: auooauooe eeoa   eoiouu liiel eeooul u el aual:: li aotou eoaloeooo:eo:oaooiai u oeaoeo aieaaolaoiooioaeei  a oaei  oeu  iuob:o:oooeoeea::u  ao  a oeoauaeoie:ioae areii aa: ooi o aori oooaaa oe  eoaeaoiuooo:oeaioeiia a uo;ui a ilaeee iiel  eeeeeui liu u oou l e aauu  oa  uaeoeeu:ouaauui:: ouiou: auoe u aaeaioaoie a oieeoiu e a ioeaoi o ou iaeeeea:: uio:lo:iaaooioul oieaeeou ioireee reo a iueeaoioo: ooa  ea

KeyboardInterrupt: 